In [1]:
import os
import sys
from datetime import datetime
from os.path import abspath
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import quantstats as qs
from hurst import compute_Hc

yf.pdr_override()

# Zipline imports
from zipline.utils.run_algo import load_extensions
from zipline.data import bundles
from zipline.data.data_portal import DataPortal
from zipline.utils.calendar_utils import get_calendar

from zipline.api import set_max_leverage, schedule_function, set_benchmark,set_commission
from zipline.finance.commission import PerContract, PerDollar, PerShare, PerTrade
from zipline.finance.commission import CommissionModel
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage
from zipline.finance.commission import PerShare, PerTrade, PerDollar
from zipline.api import set_slippage, set_commission
from zipline.data.bundles import register, unregister, ingest
from zipline.data.bundles.csvdir import csvdir_equities
from zipline.utils.calendar_utils import register_calendar, get_calendar
from zipline.api import (order, 
                         order_target,
                         order_value,
                         record, 
                         symbol,
                         get_datetime,
                         order_target_percent,
                         order_target_value,
                         set_benchmark,
                         get_open_orders)
from zipline import run_algorithm
from zipline.utils.calendar_utils import get_calendar
from zipline.api import order_target, record, date_rules, time_rules, symbol # type: ignore

# Ignore Warnings  
import warnings
warnings.filterwarnings('ignore', category=Warning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

load_extensions(
    default=True,
    extensions=[],
    strict=True,
    environ=os.environ,
)
%matplotlib inline
%load_ext autoreload
%autoreload 2

ROOT_DIR = abspath('../')
sys.path.append(ROOT_DIR)

# Todo list
1. Filter out stocks with with hurst exponents (It is good at finding mean reverting time series)
2. Built a Bollinger bands strategy
3. Use QuantStats to evaluate the Strategy

In [2]:
ROOT_DIR = abspath('../')
sys.path.append(ROOT_DIR)
TICKER_FILE_PATH = f"{ROOT_DIR}/data/sp500_tickers.csv" 
PARQUET_FILE_PATH = f"{ROOT_DIR}/data/sp500_stock_data.parquet"

START_DATE = datetime(2018, 1, 1)
END_DATE = datetime(2023, 12, 31)

def calculate_years() -> int:
    return int((END_DATE - START_DATE).days / 365)

YEARS = calculate_years()

In [3]:
def load_tickers():
    sp500_df = pd.read_csv(TICKER_FILE_PATH)
    sp500_stock_data = pd.read_parquet(PARQUET_FILE_PATH) 
    return sp500_df['Ticker'].tolist(), sp500_stock_data

def is_not_null(close_data: pd.Series) -> bool:
    years = calculate_years()
    return len(close_data) >= 251.5 * years

def calculate_hurst_exponent(close_data: pd.Series) -> float: 
    H, c, data = compute_Hc(close_data, kind="price", simplified=True)
    return H

def hurst_filter(threshold: float = 0.5): # threshold < 0.5 --> mean reverting time series
    sp500_tickers, sp500_stock_data = load_tickers()
    filtered_tickers = []
    for ticker in sp500_tickers:
        if ticker in sp500_stock_data:
            close_data = sp500_stock_data[ticker][START_DATE : END_DATE][
                "close"
            ].dropna()
            if is_not_null(close_data):
                hurst_exponent = calculate_hurst_exponent(close_data)
                if hurst_exponent <= threshold:
                    filtered_tickers.append((ticker, hurst_exponent))

    filtered_tickers.sort(key=lambda x: x[1])
    top_n_tickers = [ticker for ticker, _ in filtered_tickers[:10]]
    # top_n_tickers = pd.DataFrame(top_n_tickers)
    return top_n_tickers


def calculate_bollinger_bands(prices, window=20, num_std=2):
    rolling_mean = prices.rolling(window=window).mean()
    rolling_std = prices.rolling(window=window).std()

    upper_band = rolling_mean + num_std * rolling_std
    lower_band = rolling_mean - num_std * rolling_std

    return upper_band, rolling_mean, lower_band
    

In [4]:
def plots(results):
    start = results.index[0]
    end = results.index[-1]
    benchmark = pdr.get_data_yahoo('^GSPC', start=start, end=end)['Adj Close'].pct_change()
    results.index = pd.to_datetime(results.index).tz_convert(None)
    results.index = benchmark.index  
    qs.reports.full(results['returns'], benchmark = benchmark, match_dates=True, figsize=(8, 4))
    
    
def rebalance_static(context, data):
    for stocks in context.portfolio.positions:
        current_price = data.current(stocks, 'price')
        cost_basis = context.portfolio.positions[stocks].cost_basis
        amount = context.portfolio.positions[stocks].amount

        if amount > 0:
            if current_price * (1 + context.take_profit) > cost_basis:
                order_target_percent(stocks, 0)
                context.profits += 1
            elif current_price * (1 - context.stop_loss) < cost_basis:
                order_target_percent(stocks, 0)
                context.stops += 1
        elif amount < 0:
            if  current_price * (1 - context.take_profit) < cost_basis:
                order_target_percent(stocks, 0)
                context.profits += 1
            elif current_price * (1 + context.stop_loss) > cost_basis:
                order_target_percent(stocks, 0)
                context.stops += 1
                

In [5]:
START_DATE = pd.Timestamp('2018-01-01')
END_DATE = pd.Timestamp('2023-12-31')

BASE_CAPITAL = 100_000

In [12]:
def initialize(context):
    context.idx = 0
    context.tickers = hurst_filter()
    context.bollinger_window = 20
    context.bollinger_dev = 1.5
    context.stop_loss = 0.05 
    context.take_profit  = 0.05
    context.stops, context.profits = 0, 0
    context.buy_stocks = set()
    context.cash_pct = 0.05
    context.symbols = None
    context.sample_asset = symbol('AAPL')
    
    set_commission(PerShare(cost=0.003))
    set_slippage(VolumeShareSlippage(volume_limit=0.10, price_impact=0.15))

def handle_data(context, data):
    context.idx += 1
    if context.idx < context.bollinger_window:
        return
    
    current_date = data.history(context.sample_asset, 'price', 2, '1d').index[-1]
    context.symbols = [symbol(assets) for assets in context.tickers]
    prices = data.history(context.symbols, 'price', context.bollinger_window, '1d')
    up, mid, low = calculate_bollinger_bands(prices, context.bollinger_window, context.bollinger_dev)

    lower_band = pd.Series(np.dot(context.values, low.T), index = prices.index)
    upper_band = pd.Series(np.dot(context.values, up.T), index = prices.index)
    
    pval = pd.Series(np.dot(context.values, prices.T), index= prices.index)     # Portfolio Value
          
    if pval[current_date] < lower_band[current_date]:   # BUY Signal
        for asset in context.tickers:
            asset = symbol(asset)
            order_target_percent(asset, 2)
            
    if pval[current_date] < upper_band[current_date]:   # SELL Signal
        for asset in context.tickers:
            asset = symbol(asset)
            order_target_percent(asset, 0)
                
    # # Record the values for later analysis
    # record(
    #     price=current_price,
    #     mid=sma,
    #     upper=upper_band,
    #     lower=lower_band,
    #     stop_loss=stop_loss_price,
    #     take_profit=take_profit_price
    # )
        
# Run the algorithm
results = run_algorithm(
    start=START_DATE,
    end=END_DATE,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=BASE_CAPITAL,
    benchmark_returns=None,
    data_frequency='daily',
    bundle='sp500bundle',
)
plots(results)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [14]:

def initialize(context):
    context.idx = 0
    context.tickers = hurst_filter()
    context.bollinger_window = 20
    context.bollinger_dev = 1.5
    context.cash_pct = 0.05
    context.symbols = [symbol(assets) for assets in context.tickers]
    
    sset_commission(PerShare(cost=0.003))
    set_slippage(VolumeShareSlippage(volume_limit=0.10, price_impact=0.15))

    # Schedule rebalance function to run every trading day
    schedule_function(rebalance, date_rules.every_day(), time_rules.market_open())

def handle_data(context, data):
    context.idx += 1
    if context.idx < context.bollinger_window:
        return

    current_date = data.history(context.symbols[0], 'price', 2, '1d').index[-1]
    prices = data.history(context.symbols, 'price', context.bollinger_window, '1d')
    up, mid, low = calculate_bollinger_bands(prices, context.bollinger_window, context.bollinger_dev)

    # Check if the current portfolio value is below the lower band - Buy Signal
    if context.portfolio.portfolio_value < low.iloc[-1]:
        target_weight = 1.0 / len(context.symbols)
        for asset in context.symbols:
            order_target_percent(asset, target_weight)

    # Check if the current portfolio value is above the upper band - Sell Signal
    elif context.portfolio.portfolio_value > up.iloc[-1]:
        for asset in context.symbols:
            order_target_percent(asset, 0)

    # Record values for later analysis
    record(price=prices.iloc[-1][0], mid=mid.iloc[-1][0], upper=up.iloc[-1][0], lower=low.iloc[-1][0])

def rebalance(context, data):
    # Rebalance the portfolio at the beginning of each trading day
    total_cash = context.portfolio.cash + context.portfolio.positions_value

    for asset in context.symbols:
        if data.can_trade(asset):
            order_target_percent(asset, context.cash_pct / len(context.symbols) / data.current(asset, 'close'))

    # Print some information for debugging
    print("Rebalanced at {}: Portfolio Value: {:.2f}".format(data.current_dt, total_cash))

# Run the algorithm
results = run_algorithm(
    start=pd.Timestamp('2022-01-01'),
    end=pd.Timestamp('2022-12-31'),
    initialize=initialize,
    handle_data=handle_data,
    capital_base=100000,
    benchmark_returns=None,
    data_frequency='daily',
    bundle='sp500bundle',
)

# Display the results
print(results)

NameError: name 'commission' is not defined